# Number classification using Mnist dataset

## 1. Import libraries

In [3]:
import torch # Pytorch
import torch.nn as nn # Pytorch's neural network module

# Add the models directory to the path
# import sys
# sys.path.append("./models")

# from models import VGG16 # VGG16 model

# print(VGG16)

In [6]:
# Check GPU compatibility with Pytorch and print out GPU information
if torch.cuda.is_available():
    print("GPU is available")
    print("GPU device:", torch.cuda.get_device_name(0))
    print("GPU memory:", round(torch.cuda.get_device_properties(0).total_memory/1024**3),"GB")
else:
    print("CPU")

CPU
